# Generating new behaviors

In [ ]:
from jinja2 import Template
from pathlib import Path
import stringcase
import pandas as pd

In [ ]:
HERE = Path.cwd()
ROOT = HERE.parent.parent
BEHAVIORS_PY = ROOT  / "src/ipyforcegraph/behaviors.py"
BEHAVIORS_TS = ROOT / "js/widgets/behaviors"

In [ ]:
assert BEHAVIORS_TS.exists()
assert BEHAVIORS_PY.exists()

In [ ]:
py_cls_template = Template("""
@W.register
class {{ py_class }}(Behavior):
    _model_name: str = T.Unicode("{{ js_class }}").tag(sync=True)

    column_name: str = T.Unicode(
        None,
        allow_none=True,
        help="name of the source column to use for {{ readable }}.",
    ).tag(sync=True)

    template: Optional[str] = T.Unicode(
        None,
        allow_none=True,
        help="a nunjucks template to use to calculate {{ readable }}",
    ).tag(sync=True)
""".strip())

In [ ]:
js_class_filename = Template("""{{ kebab }}.ts""") 
js_class_template = Template("""
/*
 * Copyright (c) 2023 ipyforcegraph contributors.
 * Distributed under the terms of the Modified BSD License.
 */
import { IBehave, ILinkBehaveOptions } from '../../tokens';

import { LinkColumnOrTemplateModel } from './base';

export class {{ js_class }} extends LinkColumnOrTemplateModel implements IBehave {
  static model_name = '{{ js_class }}';

  defaults() {
    return { ...super.defaults(), _model_name: {{ js_class }}.model_name };
  }

  {{ api_method }}(options: ILinkBehaveOptions): string | null {
    return super.getLinkAttr(options);
  }
}
""".strip())

In [ ]:
token_ts_template = Template("""
  {{ api_method }}?(options: ILinkBehaveOptions): string | null;
""".strip())

In [ ]:
two_d_ts_post_update_template = Template("""
graph.{{ raw_attr }}(this.wrapFunction(this.{{ api_method }}));
""".strip())

In [ ]:
two_d_ts_method_template = Template("""
protected {{ api_method }} = (link: LinkObject): string => {
  return this.getComposedLinkAttr(link, '{{ api_method }}', '');
};
""")

In [ ]:
index_ts_template = Template("""
export * from './{{ kebab }}';
""".strip())

In [ ]:
nb_template = Template("""

### `{{ py_class }}`

add_{{ snake }} = make_link_behavior_with_ui(B.{{ py_class }}, "{{ readable }}", "{{ column_name }}"{{ is_color }})

if __name__ == "__main__":
    add_{{ snake }}(fg, box)
    display(box)

""")

In [ ]:
link_attrs = sorted(set("""
linkDirectionalArrowLength
linkDirectionalArrowLength
linkDirectionalArrowColor
linkDirectionalArrowColor
linkDirectionalArrowRelPos
linkDirectionalArrowRelPos
linkDirectionalParticles
linkDirectionalParticles
linkDirectionalParticleSpeed
linkDirectionalParticleSpeed
linkDirectionalParticleWidth
linkDirectionalParticleWidth
linkDirectionalParticleColor
linkDirectionalParticleColor
""".strip().splitlines()))
link_attrs

In [ ]:
def template_one(raw_attr):
    context = dict(
        raw_attr=raw_attr,
        py_class=f"""{raw_attr[0].upper()}{raw_attr[1:]}""",
        readable=stringcase.sentencecase(raw_attr).lower().replace(" rel ", " relative ").replace(" pos", " position"),
        kebab=stringcase.spinalcase(raw_attr).lower(),
        snake=stringcase.snakecase(raw_attr).lower(),
        is_color=", is_color=True" if "Color" in raw_attr else "",
        column_name="color" if "Color" in raw_attr else "value",

    )
    context["js_class"] = f"""{context["py_class"]}Model"""
    context["api_method"] = f"""get{context["py_class"]}"""

    result = {    
        **context,
        "js_class_filename": js_class_filename.render(**context),
        "js_class": js_class_template.render(**context),
        "py_cls_frag": py_cls_template.render(**context),
        "tokens_ts": token_ts_template.render(**context),
        "two_d_ts_post_update": two_d_ts_post_update_template.render(**context),
        "two_d_ts_method": two_d_ts_method_template.render(**context),
        "index_ts": index_ts_template.render(**context),
        "nb": nb_template.render(**context),
    }
    
    return result

In [ ]:
df = pd.DataFrame([template_one(raw_attr) for raw_attr in link_attrs])
df.T

write files

In [ ]:
# df.apply(lambda x: (BEHAVIORS_TS / x.js_class_filename).write_text(x.js_class), axis=1)

import files

In [ ]:
print("// index.ts\n")
print("".join(list(df.index_ts)))

In [ ]:
print("// tokens.ts\n")
print("\n".join(list(df.tokens_ts)))

print("' | '".join(list(df.api_method)))

In [ ]:
print("// 2d.ts\n")
print("\n".join(list(df.two_d_ts_method)))

print("\n".join(list(df.two_d_ts_post_update)))

In [ ]:
print("# behaviors.py\n")
print("\n\n".join(list(df.py_cls_frag)))

In [ ]:
print("# Behaviors.ipynb\n")
print("\n\n".join(list(df.nb)))